모듈 및 라이브러리 불러오기

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 25.7 MB/s eta 0:00:00


In [2]:
!pip install transformers pytorch_lightning tokenizers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import random
import os
import re
import torch
import tensorflow as tf

from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences
from transformers import AutoModelForSequenceClassification, pipeline

import torch.nn as nn # 분류 헤드 수정 - KoElectra 모델의 출력층을 6개의 클래스로 분류할 수 있도록 수정

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


감정분류 모델 불러오기

In [ ]:
'''# parameter
max_len = 64
batch_size = 32'''

'# parameter\nmax_len = 64\nbatch_size = 32'

In [5]:
import torch
import numpy as np
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch.nn.functional as F

# Electra 모델과 토크나이저 로드
model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer_electra = ElectraTokenizer.from_pretrained(model_name)
model_electra = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=6)  # 감정 종류가 6개라고 가정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH ='/content/drive/MyDrive/0000/Class/gradu_pro/model/model_KoElectra_0527.pt'
model_electra.load_state_dict(torch.load(PATH, map_location=device))
model_electra.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [29]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# KoGPT 모델과 토크나이저 로드
kogpt_model_name = "skt/kogpt2-base-v2"  # KoGPT 모델 이름
tokenizer_kogpt = PreTrainedTokenizerFast.from_pretrained(kogpt_model_name)
model_kogpt = GPT2LMHeadModel.from_pretrained(kogpt_model_name)
model_kogpt.to(device)

PATH ='/content/drive/MyDrive/0000/Class/gradu_pro/model/kogpt2_chatbot_model_0616.pth'
model_kogpt.load_state_dict(torch.load(PATH, map_location=device))
model_kogpt.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [18]:
# 감정 분류 함수
def classify_emotion(text):
    model_electra.eval()

    # 텍스트 클랜징 함수 : 특수문제 제거 및 여러개의 공백 하나로 줄이기
    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 입력 문장 토큰화
    inputs = tokenizer_electra(
        text,
        return_tensors="pt",
        max_length=64,  # 최대 길이 설정
        padding="max_length",
        truncation=True
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 예측
    with torch.no_grad():
        outputs = model_electra(**inputs)
        logits = outputs.logits

    # 소프트맥스 함수로 확률 계산
    probs = F.softmax(logits, dim=1)

    # 결과 해석
    probs = probs.detach().cpu().numpy()[0]
    pred_label = np.argmax(probs)

    emotion_labels = ["기쁨", "당황", "분노", "불안", "상처", "슬픔"]
    pred_emotion = emotion_labels[pred_label]
    # 각 감정에 대한 확률을 퍼센트로 변환하여 출력
    # for i, (label, prob) in enumerate(zip(emotion_labels, probs)):
    #    print(f"{label}: {prob * 100:.2f}%")

    # 가장 높은 확률을 차지하는 감정 출력
    #print(f"\n>> 입력하신 내용에서 가장 느껴지는 감정은 '{pred_emotion}'이며, 확률은 {probs[pred_label] * 100:.2f}%입니다.")

    return pred_emotion

In [26]:
# 입력 문장 예측 함수
def predict(text):
    model_electra.eval()

    # 텍스트 클랜징 함수 : 특수문제 제거 및 여러개의 공백 하나로 줄이기
    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    print(text)
    # 입력 문장 토큰화
    inputs = tokenizer_electra(
        text,
        return_tensors="pt",
        max_length=64,  # 최대 길이 설정
        padding="max_length",
        truncation=True
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 예측
    with torch.no_grad():
        outputs = model_electra(**inputs)
        logits = outputs.logits

    # 소프트맥스 함수로 확률 계산
    probs = F.softmax(logits, dim=1)

    # 결과 해석
    probs = probs.detach().cpu().numpy()[0]
    pred_label = np.argmax(probs)

    emotion_labels = ["기쁨", "당황", "분노", "불안", "상처", "슬픔"]
    pred_emotion = emotion_labels[pred_label]

    # 각 감정에 대한 확률을 퍼센트로 변환하여 출력
    for i, (label, prob) in enumerate(zip(emotion_labels, probs)):
        print(f"{label}: {prob * 100:.2f}%")

    # 가장 높은 확률을 차지하는 감정 출력
    print(f"\n>> 입력하신 내용에서 가장 느껴지는 감정은 '{pred_emotion}'이며, 확률은 {probs[pred_label] * 100:.2f}%입니다.")


생성형 공감 반응

In [56]:
# 공감형 반응 생성 함수
def generate_response(emotion, text, model=model_kogpt, tokenizer=tokenizer_kogpt, max_length=50, num_return_sequences=1):
    emotion_text = {
        "기쁨": "정말 기쁜 소식이네요! 함께 기뻐요! ",
        "슬픔": "정말 슬픈 일이네요. 함께 슬픔을 나눌게요. ",
        "분노": "그런 일이 있었다니 정말 화가 나겠어요. ",
        "불안": "걱정이 많이 되시겠어요. 함께 이겨낼 수 있을 거예요. ",
        "당황": "갑작스러운 상황이었나 봐요. 저도 당황스러워요. ",
        "상처": "마음이 많이 아프시겠어요. 위로의 말씀을 드려요. ",
        "기타": "그렇군요. 말씀해주셔서 감사합니다. "
    }

    input_text = emotion_text[emotion]
    input_ids = tokenizer_kogpt.encode(text, return_tensors='pt').to(device)

    # 모델을 사용하여 텍스트 생성
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        early_stopping=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    generated_text = tokenizer_kogpt.decode(output[0], skip_special_tokens=True)
    generated_text = input_text + generated_text
    return generated_text

In [36]:
text = '어제 금쪽이가 너무 울어서 남편과 나 모두 제대로 잠을 자지 못했다. 한명이 번갈아가며 봐줬어야 했나, 무튼 너무나도 피곤하다'
predict(text)

어제 금쪽이가 너무 울어서 남편과 나 모두 제대로 잠을 자지 못했다 한명이 번갈아가며 봐줬어야 했나 무튼 너무나도 피곤하다
기쁨: 0.09%
당황: 1.21%
분노: 1.97%
불안: 4.61%
상처: 0.54%
슬픔: 91.58%

>> 입력하신 내용에서 가장 느껴지는 감정은 '슬픔'이며, 확률은 91.58%입니다.


In [57]:
text = '오늘 갑자기 날씨가 추워져서 놀랐어'
generate_response('슬픔', text)

'정말 슬픈 일이네요. 함께 슬픔을 나눌게요. 오늘 갑자기 날씨가 추워져서 놀랐어요 '